In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import dagshub
import mlflow
import mlflow.sklearn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [3]:
train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
df = train_transaction.merge(train_identity, on='TransactionID', how='left')

In [4]:
dagshub.init(repo_owner='gioeba', repo_name='IEEE-CIS-Fraud-Detection', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=7d807ab0-aec8-4330-b3d9-f38582b45444&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=08831228bd1a4f02c8d2699d14abe1e703a3adde078be8371dd76e5dfbebadd1




Output()

Accessing as gioeba

Initialized MLflow to track repo "gioeba/IEEE-CIS-Fraud-Detection"

Repository gioeba/IEEE-CIS-Fraud-Detection initialized!

# Cleaning

In [5]:
y = df['isFraud']
X = df.drop(columns=['isFraud', 'TransactionID']).copy()

# Feature Engineering

In [6]:
logged_fe = 'runs:/2251e67e9c084b99b4e492e146e576a0/fe'
feature_engineering_model = mlflow.sklearn.load_model(logged_fe)

# Feature Selection

In [7]:
logged_preprocessor = 'runs:/de686b00b14a4c7aac22431740c0ebb5/pre-processor'
dynamic_preprocessor = mlflow.sklearn.load_model(logged_preprocessor)

In [8]:
from sklearn.feature_selection import SelectFromModel

clf_pipeline = Pipeline([
    ('feature_engineering', feature_engineering_model),
    ('preprocessing', dynamic_preprocessor),
    ('feature_select', SelectFromModel(
        DecisionTreeClassifier(
            max_depth=10,
            class_weight='balanced',
            random_state=42
        ),
        threshold="1.5*mean"
    )),
    ('classifier', DecisionTreeClassifier(max_depth=10, class_weight='balanced', random_state=42))
])

# Training

In [9]:
mlflow.set_experiment("ieee-decision-tree")
with mlflow.start_run(run_name="tree-depth-10-fs"):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    auc_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(df, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
        clf_pipeline.fit(X_train, y_train)
        y_prob = clf_pipeline.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_prob)
        auc_scores.append(auc)
    
        print(f"Fold {fold} AUC: {auc:.4f}")
        mlflow.log_metric(f"fold_{fold}_auc", auc)
    
    avg_auc = np.mean(auc_scores)
    mlflow.log_metric("avg_roc_auc", avg_auc)
    mlflow.log_param("model", "DecisionTree")
    mlflow.log_param("max_depth", 10)
    mlflow.sklearn.log_model(clf_pipeline, "model")
    print(f"\nAverage AUC across folds: {avg_auc:.4f}")

Fold 1 AUC: 0.8523
Fold 2 AUC: 0.8557
Fold 3 AUC: 0.8614
Fold 4 AUC: 0.8529
Fold 5 AUC: 0.8481


2025/04/26 18:08:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Average AUC across folds: 0.8541
🏃 View run tree-depth-10-fs at: https://dagshub.com/gioeba/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/0/runs/eb66cbd9d25c416abdb1f0abc32f7529
🧪 View experiment at: https://dagshub.com/gioeba/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/0
